In [ ]:
import re
import csv
import docx
import numpy as np

from docx import Document

In [ ]:
rowset = []
with open('HUKI for prosesser i Feiring Bruk v2.csv', 'rb') as csvfile:
    dialect = csv.Sniffer().sniff(csvfile.read(1024))
    csvfile.seek(0)
    reader = csv.reader(csvfile, dialect)
    
    rowset = []
    i = 0
    for row in reader:
        if i < 10: rowset.append(row)
        i += 1
        
D = np.array(rowset,dtype=np.string_)

    # ... process CSV file contents here ...

In [ ]:
# Koordinater
row_avd        = 2
row_rolle      = 3
row_data_start = 5

col_underaktivitet = 3 
col_data_start     = 4
col_data_end       = 4 + 8
#col_data_end       = 60

rolle = D[row_rolle, col_data_start:col_data_end+1]
desc  = D[row_data_start:,col_underaktivitet]
data  = D[row_data_start:,col_data_start:col_data_end+1]

(m,n) = data.shape

ansvar = ['H','U','K','I']
ansvartext = {'H': 'Rollen er hovedansvarlig for at følgende aktiviteter blir utført, og for å ta beslutninger knyttet til disse aktivitetene',
              'U': 'Rollen skal utføre følgende aktiviteter',
              'K': 'Rollen skal være tilgjengelig for konsultasjon i følgende aktiviteter',
              'I': 'Rollen skal informeres om følgende aktiviteter'} 
    

In [ ]:
def stringprep(s):
    return unicode(s.decode('utf-8'))

def stringclean(s):
    tmp = re.search('([a-zA-z0-9 ]*)',doc['rolle'])
    return tmp.group(0)
    

def print_document(doc):
    if doc['ansvar']:
        print '\n--------', doc['rolle'], '--------'
        for a in [aa for aa in ansvar if aa in doc['ansvar']]:
            print '\n--', a, '--'
            for d in doc['ansvar'][a].values():
                print ' * ', d['desc']
                if a=='U':
                    if d['konsultere']: print '    Konsultere:', ','.join(d['konsultere'])
                    if d['informere']:  print '    Informere: ', ','.join(d['informere'])
                    
    pass

def generate_document(doc):
    
    document = Document('blank.docx')
    
    print stringprep(doc['rolle'])
    
    #document.add_heading(, 0)
    document.add_paragraph(stringprep(doc['rolle']), style='Heading 1')

    document.add_paragraph(u'Ansvar og myndighet', style='Heading 2')
    
    for a in ansvar:
        
        document.add_paragraph(stringprep(ansvartext[a]), style='Heading 3')
        
        if a in doc['ansvar']:
            
            if a=='U':
                
                table = document.add_table(rows=1, cols=3, style='Table Grid BE-style')
                
                hdr = table.rows[0].cells
                hdr[0].text = stringprep('Skal utføre følgende aktiviteter:')
                hdr[1].text = stringprep('Skal konsultere følgende roller:')
                hdr[2].text = stringprep('Skal informere følgende roller:')
                for d in doc['ansvar']['U'].values():
                    row = table.add_row().cells
                    row[0].text = stringprep(d['desc'])
                    row[1].text = stringprep(', '.join(d['konsultere']))
                    row[2].text = stringprep(', '.join(d['informere']))
                
            else:
                for d in doc['ansvar'][a].values():
                    document.add_paragraph(stringprep(d['desc']), style='Bullet 1')
        else:
            document.add_paragraph(stringprep('Ingen aktiviteter'))
            
    
    document.save(stringclean(doc['rolle'])+u'.docx')
    
    pass     
        
        
        
        
    

In [ ]:


for j, rl in enumerate(rolle):
    doc = {'rolle': rl, 'ansvar': {}}
    for a in ansvar:
        dlist = dict(zip([i for i in xrange(m) if a in data[i,j]], [{'desc': desc[i]} for i in xrange(m) if a in data[i,j]]))
        for i in dlist:
            dlist[i]['konsultere'] = [rolle[k] for k in xrange(n) if 'K' in data[i,k]]
            dlist[i]['informere']  = [rolle[k] for k in xrange(n) if 'I' in data[i,k]]
        if dlist: doc['ansvar'][a] = dlist
    
    generate_document(doc)
    

In [ ]:
doc['ansvar']['U']